In [1]:
import io 
import pandas as pd
from collections import defaultdict
from surprise import Dataset
from surprise import NormalPredictor
from surprise import SVD
from surprise import KNNBasic
from surprise import get_dataset_dir
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split

Данные

In [2]:
data = Dataset.load_builtin('ml-100k')

Определяем алгоритмы

In [3]:
algs = {
    "normal": NormalPredictor(),
    "KNN_cos": KNNBasic(30, sim_options={'name': 'cosine'}),
    "KNN_msd": KNNBasic(30, sim_options={'name': 'msd'}),
    "KNN_pear": KNNBasic(30, sim_options={'name': 'pearson_baseline'}),
    "SVD": SVD()
}

In [4]:
algPerf = {}
for name, alg in algs.items():
    crv = cross_validate(alg, data, measures=['RMSE'], verbose=True)
    algPerf[name] = round(crv['test_rmse'].mean(), 3)

Evaluating RMSE of algorithm NormalPredictor on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5200  1.5103  1.5218  1.5222  1.5126  1.5173  0.0049  
Fit time          0.10    0.12    0.12    0.12    0.12    0.12    0.01    
Test time         0.12    0.16    0.16    0.11    0.16    0.14    0.02    
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.0109  1.0215  1.0235  1.0182  1.0224  1.0193  0.0046  
Fit time          0.83    0.97    0.88    0.88    0.86 

Поиск лучшего алгоритма

In [5]:
algPerf

{'normal': 1.517,
 'KNN_cos': 1.019,
 'KNN_msd': 0.976,
 'KNN_pear': 1.0,
 'SVD': 0.936}

In [6]:
bestAlg = {}
bestAlgName = min(algPerf,key=algPerf.get);
bestAlg = algs[bestAlgName]

bestAlgName

'SVD'

In [7]:
trainset, testset = train_test_split(data, test_size=.25)

In [15]:
bestAlg.fit(trainset)
predictn = bestAlg.test(testset)

In [9]:
#https://github.com/NicolasHug/Surprise/blob/master/examples/precision_recall_at_k.py

def precision_recall_at_k(predictions, k=10, threshold=3.5):
    """Return precision and recall at k metrics for each user"""

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        # When n_rec_k is 0, Precision is undefined. We here set it to 0.

        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 0

        # Recall@K: Proportion of relevant items that are recommended
        # When n_rel is 0, Recall is undefined. We here set it to 0.

        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 0

    return precisions, recalls

precisions, recalls = precision_recall_at_k(predictn, k=5, threshold=3.52)


precision_at_k = sum(prec for prec in precisions.values()) / len(precisions)
recall_at_k = sum(rec for rec in recalls.values()) / len(recalls)

print(precision_at_k)
print(recall_at_k)

0.7420113114174616
0.3621791611491127


In [10]:
userID = '26'
pred = list(filter(lambda x: x.uid == userID, predictn))
bestPred = sorted(pred, key=lambda x: x.est, reverse=True)[:5]

bestPred

[Prediction(uid='26', iid='50', r_ui=4.0, est=3.847770277320404, details={'was_impossible': False}),
 Prediction(uid='26', iid='181', r_ui=4.0, est=3.692657026736386, details={'was_impossible': False}),
 Prediction(uid='26', iid='302', r_ui=5.0, est=3.64938416149145, details={'was_impossible': False}),
 Prediction(uid='26', iid='276', r_ui=4.0, est=3.5892626777242778, details={'was_impossible': False}),
 Prediction(uid='26', iid='14', r_ui=3.0, est=3.409254834624297, details={'was_impossible': False})]

Получаем фильмы

In [16]:
import os
filePath = os.path.expanduser('~/.surprise_data/ml-100k/ml-100k/u.item')
films = pd.read_csv(filePath, sep="|", encoding='latin-1', header=None)

In [12]:
data = {x.iid:x.est for x in bestPred}
data

{'50': 3.847770277320404,
 '181': 3.692657026736386,
 '302': 3.64938416149145,
 '276': 3.5892626777242778,
 '14': 3.409254834624297}

Результат

In [13]:
print("User "+str(userID))
for [filmId,est] in data.items():
    filmData = list(films.iloc[int(filmId),:4].dropna().values)
    filmData.append(est)
    print(str(filmData))

User 26
[51, 'Legends of the Fall (1994)', '01-Jan-1994', 3.847770277320404]
[182, 'GoodFellas (1990)', '01-Jan-1990', 3.692657026736386]
[303, "Ulee's Gold (1997)", '01-Jan-1997', 3.64938416149145]
[277, 'Restoration (1995)', '01-Jan-1995', 3.5892626777242778]
[15, "Mr. Holland's Opus (1995)", '29-Jan-1996', 3.409254834624297]
